**Min-Max with depth 6**

In [1]:
grid = [
    [1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1],
    [1,5,5,5,1,5,5,5,1,1],
    [1,5,5,5,1,5,5,5,1,1],
    [1,5,5,5,1,5,5,5,1,1],
    [1,0,0,0,0,0,0,0,0,1]
]

#function to display grid
def display(pos_dict):
    
    for i in range(0,6):
        print(""+str(pos_dict[(i,0)])+"-"+str(pos_dict[(i,1)])+"-"+str(pos_dict[(i,2)])+"-"+str(pos_dict[(i,3)])+"-"+str(pos_dict[(i,4)])+"-"+str(pos_dict[(i,5)])+"-"+str(pos_dict[(i,6)])+"-"+str(pos_dict[(i,7)])+"-"+str(pos_dict[(i,8)])+"-"+str(pos_dict[(i,9)]))

pos_dict = {}
backtracking_dict = {}

for i in range(0,10):
    for j in range(0,6):
        pos_dict[(j,i)] = 0

goal_position = (5,9)
blue_position = (5,0)
red_position = (0,4)

#defines final position where the search stopped, in order to backtrack incase the search never got completed
final_position = (5,0)


def neighbors(position,temp_red, temp_blue):
    lst=[(position[0],position[1]+1),(position[0],position[1]-1),(position[0]-1,position[1]),(position[0]+1,position[1])]
    answer=[]
    for p in lst:
        if(p == temp_blue):
            continue
        elif(p == temp_red):
            continue
        elif( (p[0] < 0 or p[0] > 5 or p[1] < 0 or p[1] > 9) or (p[0] == 5 and (p[1] >= 1 and p[1] <= 8))):
            continue
        answer.append(p)
    return answer


def get_heuristic(blue_position, red_position):
    
    mdist = abs(goal_position[0] - blue_position[0]) + abs(goal_position[1] - blue_position[1])
    vdist = abs(blue_position[0] - red_position[0])
    
    return (-mdist)+vdist

def backtrack_update(current_step, next_step):
  #later pop the 0th element to get the cost path
  #backtracking dictionary is of the form ==> key tuple: value list of tuples ==> key maze position: ordered list of its predecessors
    if next_step not in backtracking_dict.keys():
        backtracking_dict[next_step] = [current_step]
    else:
        backtracking_dict[next_step].append(current_step)
    return

def create_subtree(d,level_type, parent_blue, parent_red):
    
    if(d == 6):
        ## heuristic defined to calculate utility function
        return get_heuristic(parent_blue, parent_red)
    
    if(d < 6):
        if(level_type == "min"):
            answer = neighbors(parent_red, parent_red, parent_blue)
            temp_v = float('inf')
            temp_position = tuple()
            for p in answer:
                v = create_subtree(d+1,"max",parent_blue, p)
                if(v <= temp_v):
                    temp_v = v
                    temp_position = p
            return temp_v
        elif(level_type == "max"):
            answer = neighbors(parent_blue, parent_red, parent_blue)
            temp_v = float('-inf')
            temp_position = tuple()
            for p in answer:
                v = create_subtree(d+1,"min",p, parent_red)
                if(v >= temp_v):
                    temp_v = v
                    temp_position = p
            return temp_v

#initiating environment
pos_dict[blue_position]=1
pos_dict[red_position]=2
backtracking_dict[blue_position]=[]
print("initial maze")
display(pos_dict)
count = 0
print("Next")

def min_max():
    
    global blue_position
    global red_position
    global goal_position
    global final_position
    global count
    
    while((blue_position != goal_position) and (count < 100)):
        
        count += 1        
        answer = neighbors(blue_position, red_position, blue_position)
        if len(answer) == 1:    #Selecting the only neighbor available and no need of creating a tree for that neighbor
            #result of the min-max is in in temp varible that will be stored in backtracking dictionary to track the cost path later, if there exists one
            backtrack_update(blue_position, answer[0])
            
            #after updating backtrack dictionary, we place actual blue agent position in its global variable
            pos_dict[blue_position] = 0
            blue_position = answer[0]
            pos_dict[blue_position] = 1
        elif len(answer) >= 2:
            temp_max = float('-inf')
            temp_max_position = tuple()
            for p in answer:
                v = create_subtree(0, "min", p, red_position)  ## we get min for all subttrees and then will calculate max 
                if v >= temp_max:
                    temp_max = v
                    temp_max_position = p
            #result of the min-max is in in temp varible that will be stored in backtracking dictionary to track the cost path later, if there exists one
            backtrack_update(blue_position, temp_max_position)
            
            #after updating backtrack dictionary, we place actual blue agent position in its global variable
            pos_dict[blue_position] = 0
            blue_position = temp_max_position
            pos_dict[blue_position] = 1

        final_position = blue_position    
        if(blue_position == goal_position):
            print("Goal reached.")
            return
        
        answer = neighbors(red_position, red_position, blue_position)
        
        if len(answer) == 1:    #Selecting the only neighbor available and no need of creating a tree for that neighbor
            #after updating backtrack dictionary, we place actual red agent position in its global variable
            pos_dict[red_position] = 0
            red_position = answer[0]
            pos_dict[red_position] = 2
        elif len(answer) >= 2:
            temp_max = float('inf')
            temp_max_position = tuple()
            for p in answer:
                v = create_subtree(0, "max", blue_position, p)  ## we get min for all subttrees and then will calculate max 
                if v <= temp_max:
                    temp_max = v
                    temp_max_position = p
            #after updating backtrack dictionary, we place actual red agent position in its global variable
            pos_dict[red_position] = 0
            red_position = temp_max_position
            pos_dict[red_position] = 2
        
        display(pos_dict)
        print("Next iteration")

initial maze
0-0-0-0-2-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
1-0-0-0-0-0-0-0-0-0
Next


In [2]:
min_max()

0-0-0-0-0-0-0-0-0-0
0-0-0-0-2-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
1-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
Next iteration
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-2-0-0-0-0-0
1-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
Next iteration
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-2-0-0-0-0-0
1-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
Next iteration
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-1-0-0-2-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
Next iteration
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-1-0-0-2-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
Next iteration
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-1-0-0-2-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
Next iteration
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-1-0-0-2-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
Next iteration
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0-0-0
0-0-0-0-0-0-0-0

In [3]:
lst1=[final_position]
track_position=final_position
while(True):
  t = backtracking_dict[track_position]
  if t == []:
    break
  lst1.append(t[0])
  track_position = t[0]
lst1

[(4, 1), (4, 0), (5, 0)]

In [4]:
print("backtracking dictionary ==> (key=future-position):(value=list of predecessor postions) : ", str(backtracking_dict))
print("path from start to final position: ", str(lst1))
cost = 0
for k in lst1:
    cost += grid[k[0]][k[1]]
if count==100:
  print("Search could not reach goal and and got stuck to cost is 100 as per directions in question")
  print("The cost of path till the point it got stuck =",cost)
else:
  print("Search reached goal with path cost = ",cost)

backtracking dictionary ==> (key=future-position):(value=list of predecessor postions) :  {(5, 0): [], (4, 0): [(5, 0), (3, 0)], (3, 0): [(4, 0)], (4, 1): [(4, 0), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1), (3, 1)], (3, 1): [(4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1)]}
path from start to final position:  [(4, 1), (4, 0), (5, 